<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/RTOFS/hurr/ATL/2025/combine_forecast_incl_IC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime, timedelta

import glob as glob
import xarray as xr

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
date_start, date_end = ['20251020', '20251031']

save_data_path = '/content/drive/MyDrive/RTOFS/publications/Melissa2025_nATL/saved_data/RTOFS/'

## Add initial time (from nowcast) to the a single dataset

In [ ]:
start_date = datetime.strptime(date_start, '%Y%m%d')
end_date = datetime.strptime(date_end, '%Y%m%d')

current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime('%Y%m%d')
    fname_nowcast = f'{save_data_path}nowcast/{date_str}.nc'
    fname_forecast = f'{save_data_path}forecast/{date_str}.nc'

    ds_nowcast = xr.open_dataset(fname_nowcast)
    ds_forecast = xr.open_dataset(fname_forecast)

    ds_nowcast_filtered = ds_nowcast.isel(Depth=0)
    #ds_forecast_filtered = ds_forecast.sel(MT=ds_forecast.MT.dt.hour == 0).isel(Depth=0)
    ds_forecast_filtered = ds_forecast.isel(Depth=0)

    combined_ds = xr.concat([ds_nowcast_filtered.isel(MT=-1), ds_forecast_filtered], dim='MT')

    print(f'Processed data for {date_str}. Combined dataset has {combined_ds.sizes["MT"]} time steps.')

    fname_save = f'{save_data_path}combined/surf_{date_str}.nc'
    combined_ds.to_netcdf(fname_save)
    print(f'Saved combined dataset to {fname_save}')

    current_date += timedelta(days=1)

## Plot `SST`

In [ ]:
start_date = datetime.strptime(date_start, '%Y%m%d')
end_date = datetime.strptime(date_end, '%Y%m%d')

# Calculate the number of days to plot.
num_days = (end_date - start_date).days + 1
num_cols = 5 # As requested by the user

# Create a single figure with subplots.
fig, axes = plt.subplots(num_days, num_cols, figsize=(10, 16)) # Set the number of rows to num_days and requested figsize

current_date = start_date
day_index = 0  # Initialize index for subplot rows

while current_date <= end_date:
    date_str = current_date.strftime('%Y%m%d')
    fname_save = f'{save_data_path}combined/surf_{date_str}.nc'
    print(f'Reading processded data from\t: {fname_save}...')
    combined_ds = xr.open_dataset(fname_save)

    # Select the time steps where the hour is 0
    ds_to_plot = combined_ds.sel(MT=combined_ds.MT.dt.hour == 0)

    # Iterate through the selected time steps and plot in the corresponding column
    for col in range(num_cols):
        if col < ds_to_plot.sizes['MT']:
            # Select the appropriate subplot using the day_index and col, and plot using pcolormesh.
            # Remove individual colorbars and clear titles
            plot = ds_to_plot.isel(MT=col).sel(Y=slice(100, 375), X=slice(100, 500)).temperature.plot.pcolormesh(x='X', y='Y', vmin=24, vmax=32, cmap='Spectral_r', ax=axes[day_index, col], add_colorbar=False)
            axes[day_index, col].set_title('') # Clear the title

            # Remove xlabels and ylabels and ticks from most subplots
            if day_index < num_days - 1:
                axes[day_index, col].set_xlabel('')
                axes[day_index, col].tick_params(labelbottom=False)
            if col > 0:
                axes[day_index, col].set_ylabel('')
                axes[day_index, col].tick_params(labelleft=False)

            # Add date_str to the first column of each row
            if col == 0:
                axes[day_index, col].text(400, 375, date_str)

        else:
            # Hide unused subplots if there are fewer than 5 time steps with hour 0
            fig.delaxes(axes[day_index, col])


    day_index += 1 # Increment the index for the next row (day)
    current_date += timedelta(days=1)

# Add a single horizontal colorbar for the entire figure
cbar_ax = fig.add_axes([0.1, 0.06, 0.8, 0.02]) # [left, bottom, width, height]
cbar = fig.colorbar(plot, cax=cbar_ax, orientation='horizontal')
cbar.ax.set_xlabel('SST [$\\circ$C]')


#plt.tight_layout(rect=[0, 0.08, 1, 1]) # Adjust layout to prevent overlapping titles and labels, making space for colorbar
plt.show() # Display the plot

## Plot `SSS`

In [ ]:
start_date = datetime.strptime(date_start, '%Y%m%d')
end_date = datetime.strptime(date_end, '%Y%m%d')

# Calculate the number of days to plot.
num_days = (end_date - start_date).days + 1
num_cols = 5 # As requested by the user

# Create a single figure with subplots.
fig, axes = plt.subplots(num_days, num_cols, figsize=(10, 16)) # Set the number of rows to num_days and requested figsize

current_date = start_date
day_index = 0  # Initialize index for subplot rows

while current_date <= end_date:
    date_str = current_date.strftime('%Y%m%d')
    fname_save = f'{save_data_path}combined/surf_{date_str}.nc'
    print(f'Reading processded data from\t: {fname_save}...')
    combined_ds = xr.open_dataset(fname_save)

    # Select the time steps where the hour is 0
    ds_to_plot = combined_ds.sel(MT=combined_ds.MT.dt.hour == 0)

    # Iterate through the selected time steps and plot in the corresponding column
    for col in range(num_cols):
        if col < ds_to_plot.sizes['MT']:
            # Select the appropriate subplot using the day_index and col, and plot using pcolormesh.
            # Remove individual colorbars and clear titles
            plot = ds_to_plot.isel(MT=col).sel(Y=slice(100, 375), X=slice(100, 500)).salinity.plot.pcolormesh(x='X', y='Y', vmin=34, vmax=38, cmap='gist_ncar', ax=axes[day_index, col], add_colorbar=False)
            axes[day_index, col].set_title('') # Clear the title

            # Remove xlabels and ylabels and ticks from most subplots
            if day_index < num_days - 1:
                axes[day_index, col].set_xlabel('')
                axes[day_index, col].tick_params(labelbottom=False)
            if col > 0:
                axes[day_index, col].set_ylabel('')
                axes[day_index, col].tick_params(labelleft=False)

            # Add date_str to the first column of each row
            if col == 0:
                axes[day_index, col].text(400, 375, date_str)

        else:
            # Hide unused subplots if there are fewer than 5 time steps with hour 0
            fig.delaxes(axes[day_index, col])


    day_index += 1 # Increment the index for the next row (day)
    current_date += timedelta(days=1)

# Add a single horizontal colorbar for the entire figure
cbar_ax = fig.add_axes([0.1, 0.06, 0.8, 0.02]) # [left, bottom, width, height]
cbar = fig.colorbar(plot, cax=cbar_ax, orientation='horizontal')
cbar.ax.set_xlabel('SSS [PSU]')


#plt.tight_layout(rect=[0, 0.08, 1, 1]) # Adjust layout to prevent overlapping titles and labels, making space for colorbar
plt.show() # Display the plot